<a href="https://colab.research.google.com/github/Leonild/DS_mAIstros/blob/main/API_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API

GIT: https://github.com/Leonild/DS_mAIstros/blob/main/API_NLP.ipynb

In [ ]:
!pip install Flask-Waitress
!pip install flask-restful
!pip install tensorflow
!pip install tensorflow_hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from tensorflow.keras.models import load_model

In [ ]:
# API
# Import necessary libraries
import flask
#from waitress import serve
from flask import Flask, request, redirect, url_for, jsonify, render_template
from flask_restful import reqparse, abort, Api, Resource
import json

In [ ]:
# Set the URL for the module to load
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
# Load the embedder module from the specified URL using Tensorflow Hub
embedder = hub.load(module_url)
# Load the model named 'NLP-project-GOUSE-2-1' without compiling it
#model = load_model('/content/NLP-project-GOUSE-2-1',compile=False)
model_text = tf.saved_model.load("./NLP-project-GOUSE-2-1")
model_sentiments = tf.saved_model.load("./NLP-project-GOUSE-1-1")

## This function takes a sentence and returns its embedding

In [ ]:
def embed(sentence):
    # Wrap the sentence in a list
    sentence_in =[sentence]
    # Return the embedding of the sentence
    return embedder(sentence_in)

## This function takes a job description and predicts the most relevant technology based on its embedding

In [ ]:
def predict_label_text(JD):
    probs=model_text([JD])
    # List of technologies to predict
    techs=['News','Opinion','Trash']
    # Combine the technology names and their corresponding probabilities into a list and sort it in descending order of probability
    results=sorted(list(zip(techs,probs[0])), key=lambda x: x[1], reverse=True)
    # Return the sorted list
    return results

def predict_label_sentiments(JD):
    probs=model_sentiments([JD])
    # List of technologies to predict
    techs=['Happy','Sad','Ambiguous','Neutral']
    # Combine the technology names and their corresponding probabilities into a list and sort it in descending order of probability
    results=sorted(list(zip(techs,probs[0])), key=lambda x: x[1], reverse=True)
    # Return the sorted list
    return results

In [ ]:
def classification_text(text):
  result_text = predict_label_text(text)
  response_text = ""
  biggest = 0.0
  for i in range(len(result_text)):
    #print("Class:", result[i][0], " Probability: ", f'{result[i][1].numpy():.5f}')
    value = f'{result_text[i][1].numpy():.5f}'
    if np.float16(value) > biggest:
      response_text = result_text[i][0]
      biggest = np.float16(value)
        
  result_sentiments = predict_label_sentiments(text)
  response_sentiments = ""
  biggest = 0.0
  for i in range(len(result_sentiments)):
    #print("Class:", result[i][0], " Probability: ", f'{result[i][1].numpy():.5f}')
    value = f'{result_sentiments[i][1].numpy():.5f}'
    if np.float16(value) > biggest:
      response_sentiments = result_sentiments[i][0]
      biggest = np.float16(value)

  response = "Text type is [" + response_text + "] and the sentiment is [" + response_sentiments + "]"
  return response

print(classification_text("This is a lovely day"))

Text type is [News] and the sentiment is [Happy]


In [ ]:
from flask import Flask, request, render_template,jsonify
app = Flask(__name__)

#loading interface
@app.route('/')
def home():
    return render_template('interface.html')

# dealing with get and post
@app.route('/classify', methods=['GET','POST'])
def answerQuestion():
    question = request.form['text_input']
    classification = classification_text(question)
    return render_template('interface.html', result = classification)
    #return redirect(url_for("result", result=classification))



# Check if this file is being executed as the main program
if __name__ == '__main__':
  
    # Start the API and listen for incoming requests on all network interfaces
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
